In [25]:
pred_path='../data/processed_datasets/RICO_ltrb_lex_refine/src1_test.txt'

In [26]:
# this file is to convert generated seq format data to layout data
# input: pred data in seq format, generated by Diffusion-LM decoder
# output: pred data in layout format (lrtb), used for drawing to pics

import io

out_path='/'.join(pred_path.split('/')[0:-1])+'/processed.pt'

with open(pred_path,'r') as f:
    lines=f.readlines()

In [27]:
##sep the line

for i,line in enumerate(lines):
    line='START '+line.replace('\n','')+' END ]\n'
    words=line.split(' ')
    words_new=[word.strip('["').replace('"]\n','') for word in words]
    lines[i]=words_new

#get legal layout

layouts=[]
for line in lines:
    sos=999
    eos=0
    tmp=0
    for i,word in enumerate(line):
        if word=='START':
            sos=i
        elif word=='END':
            eos=i
    # COMMENT BELOW TO SETTING STRICT CONSTRAINED
    
        elif word=='|':     #last <sep>
            tmp=i
            if sos==999:    #only set when first time
                sos=i
    if eos==0 or eos<sos:
        eos=tmp

    # COMMENT ABOVE TO SETTING STRICT CONSTRAINED
    
    if sos<eos:
        layouts.append(line[sos+1:eos])

layouts[1]

['Image',
 '3',
 '5',
 '127',
 '26',
 '|',
 'Input',
 '2',
 '35',
 '121',
 '45',
 '|',
 'Input',
 '3',
 '47',
 '125',
 '55',
 '|',
 'Text_Button',
 '2',
 '59',
 '120',
 '66',
 '|',
 'Text_Button',
 '5',
 '66',
 '126',
 '72']

In [28]:
#separate layouts by type
failed=0
layouts_sep=[]
for list1 in layouts:
    if '|' not in list1: #only one elem
        LIST = []
        LIST.append(list1[0:5])
        layouts_sep.append(LIST)
        continue

    index_list = []   
    eee = enumerate(list1)
    for index, item in eee:
        if item == '|':
            index_list.append(index)

    index_list = [0] + index_list

    head = list1[0:index_list[1]]
    tail = list1[index_list[len(index_list)-1]:]
    tail.pop(0)
    c = len(index_list)
    LIST = []
    i = 1
    for i in range(1, c-1):    
        small_list = []
        small_list = list1[index_list[i]:index_list[i+1]]
        small_list.pop(0)
        if small_list:
            LIST.append(small_list)
        else: 
            failed+=1 #line 655
        i = i + 1

    LIST.insert(0, head)
    LIST.append(tail)
    layouts_sep.append(LIST)

print(failed)
print(len(layouts_sep),"is the number of processed layouts")

0
3729 is the number of processed layouts


In [29]:
import torch

def decapulate(bbox):
    if len(bbox.size()) == 2:
        x1, y1, x2, y2 = bbox.T
    else:
        x1, y1, x2, y2 = bbox.permute(2, 0, 1)
    return x1, y1, x2, y2

def convert_ltwh_to_ltrb(bbox):
    l, t, w, h = decapulate(bbox)
    r = l + w
    b = t + h
    return torch.stack([l, t, r, b], axis=-1)

def continuize(bbox):
    """
    Args:
        discrete_bbox torch.LongTensor: N * 4

    Returns:
        continuous_bbox torch.Tensor: N * 4
    """
    # if(len(bbox)!=4):
    #     print("Failed")
    #     return torch.stack([torch.tensor(0), torch.tensor(0), torch.tensor(0), torch.tensor(0)], dim=-1).float()
    # else:
    bbox=torch.tensor(bbox).long()
    if 'ltrb' in pred_path or 'bst' in pred_path:
        x1, y1, x2, y2 = decapulate(bbox)
    else:
        x1, y1, x2, y2 = decapulate(convert_ltwh_to_ltrb(bbox))
    # x1, y1, x2, y2 = decapulate(bbox)
    cx1, cx2 = x1 / 127, x2 / 127
    cy1, cy2 = y1 / 127, y2 / 127
    return torch.stack([cx1, cy1, cx2, cy2], dim=-1).float()

layouts_final=[]

labels_RICO = [
    'Text', 'Image', 'Icon', 'List_Item', 'Text_Button', 'Toolbar',
    'Web_View', 'Input', 'Card', 'Advertisement', 'Background_Image',
    'Drawer', 'Radio_Button', 'Checkbox', 'Multi_Tab', 'Pager_Indicator',
    'Modal', 'On_Off_Switch', 'Slider', 'Map_View', 'Button_Bar', 'Video',
    'Bottom_Navigation', 'Number_Stepper', 'Date_Picker'
]
labels_Pub = [
    'text',
    'title',
    'list',
    'table',
    'figure',
]
max=0
for layout in layouts_sep:
    layout_final={}
    layout_final['pred']=[]
    bboxs=[]
    labels=[]
    bboxs_continue=[]
    if len(layout)>max:
        max = len(layout)
    for element in layout:
        if not element:
            continue
        if type(bboxs) is not list:
            bboxs=bboxs.tolist()
        if len(element)>=4 and str.isdigit(element[-4]) and str.isdigit(element[-3]) and str.isdigit(element[-2]) and str.isdigit(element[-1]):
            bboxs.append([int(num) for num in element[-4:]])
        else:
            continue
        bboxs_continue=continuize(bboxs)

        if 'pub' in pred_path or 'Pub' in pred_path:
            # print('publaynet')
            if str.isdigit(element[1]):
                if element[0] in labels_Pub:
                    labels.append(labels_Pub.index(element[0])+1)
                else:
                    continue
        else:
            if str.isdigit(element[1]):
                if element[0] in labels_RICO:
                    labels.append(labels_RICO.index(element[0])+1)
                else:
                    continue
    if len(bboxs_continue)<len(labels):
        labels=labels[:len(bboxs_continue)]
    if len(bboxs_continue)>len(labels):
        bboxs_continue=bboxs_continue[:len(labels)]
    layout_final['pred'].append(bboxs_continue)
    layout_final['pred'].append(labels)
    layouts_final.append(layout_final)
    
print(layouts_final[15]['pred'])
print(len(layouts_final[15]['pred'][0]))
print(max)

[tensor([[0.0000, 0.0236, 0.1260, 0.0866],
        [0.0630, 0.3307, 0.9370, 0.3858],
        [0.1654, 0.0315, 0.4331, 0.0630],
        [0.0709, 0.1260, 0.8976, 0.1969],
        [0.0472, 0.8346, 0.9921, 0.8976],
        [0.0394, 0.3937, 0.9449, 0.4252],
        [0.0000, 0.0315, 0.9921, 0.1102]]), [3, 8, 1, 1, 5, 5, 6]]
7
20


In [30]:
import pickle

with open(out_path,'wb') as f:
    pickle.dump(layouts_final,f)

In [31]:
from os import path

out_path = out_path.replace('../','')

# run the following command in the root directory of this repo to draw pics

if 'pub' not in pred_path:
    print('python eval_src/tools/draw_from_results.py -d rico -p '+out_path+' -s '+'/'.join(out_path.split('/')[0:-1])+'/pics -n 4000')
else:
    print('python eval_src/tools/draw_from_results.py -d publaynet -p '+out_path+' -s '+'/'.join(out_path.split('/')[0:-1])+'/pics -n 4000')

python eval_src/tools/draw_from_results.py -d rico -p data/processed_datasets/RICO_ltrb_lex_refine/processed.pt -s data/processed_datasets/RICO_ltrb_lex_refine/pics -n 4000
